In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import os
import glob

In [12]:
from utility import predict_fuc, load_best_model, postprocessing
from preprocessing import process_and_memmap_chunks, get_index, del_files, normalize_and_reshape_chunks, preprocess
from NoiseReduction import EnhancedTimeWavelengthTransformer

In [5]:
path_folder = '/kaggle/input/ariel-data-challenge-2024/'  # path to the folder containing the data
path_out = '/kaggle/working/'  # path to the folder to store the light data
if not os.path.exists(path_out):
    os.makedirs(path_out)
    print(f"Directory {path_out} created.")
else:
    print(f"Directory {path_out} already exists.")
output_file = path_out + 'dataset_test.npy'
CHUNKS_SIZE = 1

Directory /kaggle/working/ already exists.


In [6]:
files = glob.glob(os.path.join(path_folder + 'test/', '*/*'))
files = [file.replace('\\', '/') for file in files]
index = get_index(files, CHUNKS_SIZE)  ## Adjusted to process the test set

In [ ]:
preprocess(path_folder, path_out, index, CHUNKS_SIZE=CHUNKS_SIZE)

In [7]:
# 逐块处理并合并数据
dataset_test = process_and_memmap_chunks(path_out, len(index), CHUNKS_SIZE,
                                         output_file)
# 删除临时文件
del_files(path_out + 'AIRS_clean_test', len(index))
del_files(path_out + 'FGS1_test', len(index))

In [13]:
data_test_reshaped = normalize_and_reshape_chunks(dataset_test,
                                                  len(index) * CHUNKS_SIZE)

In [4]:
batch_size = 1
model = EnhancedTimeWavelengthTransformer().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model, optimizer, best_val_loss, target_min, target_max = load_best_model(
    model, optimizer,
    '/kaggle/input/noisereduction/pytorch/default/1/best_model_NR_v2.0.pth')

Best model loaded.


In [14]:
all_predictions = predict_fuc(model, data_test_reshaped, batch_size=1)
all_predictions = all_predictions * (target_max - target_min) + target_min
all_predictions = all_predictions.numpy()

In [ ]:
wavelengths = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')
test_adc_info = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
    index_col='planet_id')

In [ ]:
sub_df = postprocessing(all_predictions,
                        wavelengths,
                        test_adc_info.index,
                        sigma_pred=np.tile(
                            np.where(test_adc_info[['star']] <= 1, 0.000171,
                                     0.001), (1, 283)))

In [ ]:
display(sub_df)

In [ ]:
sub_df.to_csv('/kaggle/working/submission.csv', index=False)